# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          30 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/29 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.09252


extracting tarball to tmp_2209.09252... done.


Retrieving document from  https://arxiv.org/e-print/2209.09253


extracting tarball to tmp_2209.09253... done.
Retrieving document from  https://arxiv.org/e-print/2209.09255


...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.09877


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.09864/Master_LiteBIRD_Detectors_SPIE_2022.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'acronyms' from 'tmp_2209.09864/acronyms.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2209.09877...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.09877/SARA_Photo_z.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09823-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09823) | **Can we detect deep axisymmetric toroidal magnetic fields in stars?**  |
|| Hachem Dhouib, et al. -- incl., <mark>Lisa Bugnet</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *4 pages, 2 figures. Proceeding of the Annual meeting of the French Society of Astronomy and Astrophysics (SF2A 2022)*|
|**Abstract**| One of the major discoveries of asteroseismology is the signature of a strong extraction of angular momentum (AM) in the radiative zones of stars across the entire Hertzsprung-Russell diagram, resulting in weak core-to-surface rotation contrasts. Despite all efforts, a consistent AM transport theory, which reproduces both the internal rotation and mixing probed thanks to the seismology of stars, remains one of the major open problems in modern stellar astrophysics. A possible key ingredient to figure out this puzzle is magnetic field with its various possible topologies. Among them, strong axisymmetric toroidal fields, which are subject to the so-called Tayler MHD instability, could play a major role. They could trigger a dynamo action in radiative layers while the resulting magnetic torque allows an efficient transport of AM. But is it possible to detect signatures of these deep toroidal magnetic fields? The only way to answer this question is asteroseismology and the best laboratories of study are intermediate-mass and massive stars because of their external radiative envelope. Since most of these are rapid rotators during their main-sequence, we have to study stellar pulsations propagating in stably stratified, rotating, and potentially strongly magnetised radiative zones. For that, we generalise the traditional approximation of rotation, which provides in its classic version a flexible treatment of the adiabatic propagation of gravito-inertial modes, by taking simultaneously general axisymmetric differential rotation and toroidal magnetic fields into account. Using this new non-perturbative formalism, we derive the asymptotic properties of magneto-gravito-inertial modes and we explore the different possible field configurations. We found that the magnetic effects should be detectable for equatorial fields using high-precision asteroseismic data. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09604-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09604) | **The JCMT BISTRO Survey: Multi-wavelength polarimetry of bright regions  in NGC 2071 in the far-infrared/submillimetre range, with POL-2 and HAWC+**  |
|| L. Fanciullo, et al. -- incl., <mark>C. W. Lee</mark>, <mark>T. Liu</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *Main article: 18 pages, 11 figures. Online supplemental material: 2 pages, 3 figures*|
|**Abstract**| Polarized dust emission is a key tracer in the study of interstellar medium and of star formation. The observed polarization, however, is a product of magnetic field structure, dust grain properties and grain alignment efficiency, as well as their variations in the line of sight, making it difficult to interpret polarization unambiguously. The comparison of polarimetry at multiple wavelengths is a possible way of mitigating this problem. We use data from HAWC+/SOFIA and from SCUBA-2/POL-2 (from the BISTRO survey) to analyse the NGC 2071 molecular cloud at 154, 214 and 850 $\mu$m. The polarization angle changes significantly with wavelength over part of NGC 2071, suggesting a change in magnetic field morphology on the line of sight as each wavelength best traces different dust populations. Other possible explanations are the existence of more than one polarization mechanism in the cloud or scattering from very large grains. The observed change of polarization fraction with wavelength, and the 214-to-154 $\mu$m polarization ratio in particular, are difficult to reproduce with current dust models under the assumption of uniform alignment efficiency. We also show that the standard procedure of using monochromatic intensity as a proxy for column density may produce spurious results at HAWC+ wavelengths. Using both long-wavelength (POL-2, 850 $\mu$m) and short-wavelength (HAWC+, $\lesssim 200\, \mu$m) polarimetry is key in obtaining these results. This study clearly shows the importance of multi-wavelength polarimetry at submillimeter bands to understand the dust properties of molecular clouds and the relationship between magnetic field and star formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09252-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09252) | **Calibration of neutron star natal kick velocities to isolated pulsar  observations**  |
|| Veome Kapil, Ilya Mandel, Emanuele Berti, <mark>Bernhard Müller</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *8 pages, 8 figures, 1 table*|
|**Abstract**| Current prescriptions for supernova natal kicks in rapid binary population synthesis simulations are based on fits of simple functions to single pulsar velocity data. We explore a new parameterization of natal kicks received by neutron stars in isolated and binary systems developed by Mandel & M\"uller, which is based on 1D and 3D supernova simulations and accounts for the physical correlations between progenitor properties, remnant mass, and the kick velocity. We constrain two free parameters in this model using very long baseline interferometry velocity measurements of Galactic single pulsars. We find that the inferred values of natal kick parameters do not differ significantly between single and binary evolution scenarios. The best-fit values of these parameters are $v_{\rm ns} = 520$ km s$^{-1}$ for the scaling pre-factor for neutron star kicks, and $\sigma_{\rm ns}=0.3$ for the fractional stochastic scatter in the kick velocities. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09695-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09695) | **Searching for axion dark matter with MeerKAT Radio Telescope**  |
|| Yun-Fan Zhou, et al. -- incl., <mark>Shao-Guang Guo</mark>, <mark>Mengtian Li</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *6 pages, 3 figures, accepted by Physical Review D*|
|**Abstract**| Axions provide a natural and well-motivated dark matter candidate, with the capability to convert directly to photons in the presence of an electromagnetic field. A particularly compelling observational target is the conversion of dark matter axions into photons in the magnetospheres of highly magnetised neutron stars, which is expected to produce a narrow spectral peak centred at the frequency of the axion mass. We point the MeerKAT radio telescope towards the isolated neutron star J0806.4$-$4123 for $10$-hours of observation and obtain the radio spectra in the frequency range $769$-$1051$ MHz. By modelling the conversion process of infalling axion dark matter (DM) we then compare these spectra to theoretical expectations for a given choice of axion parameters. Whilst finding no signal above $5\sigma$ in the data, we provide a unique constraint on the Primakoff coupling of axion DM, $g_{{\rm a}\gamma\gamma}\lesssim 9.3 \times 10^{-12}\,{\rm GeV}^{-1}$ at the $95\%$ confidence level, in the mass range $3.18$-$4.35\,\mu$eV. This result serves the strongest constraint in the axion mass range $4.20$-$4.35\,\mu$eV. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09782-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09782) | **Dark Energy Survey Year 3 results: Magnification modeling and impact on  cosmological constraints from galaxy clustering and galaxy-galaxy lensing**  |
|| J. Elvin-Poole, et al. -- incl., <mark>K. Herner</mark>, <mark>I. Harrison</mark>, <mark>E. Krause</mark>, <mark>A. R. Liddle</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *21 pages, 13 figures, See this this https URL URL for the full DES Y3 cosmology release*|
|**Abstract**| We study the effect of magnification in the Dark Energy Survey Year 3 analysis of galaxy clustering and galaxy-galaxy lensing, using two different lens samples: a sample of Luminous red galaxies, redMaGiC, and a sample with a redshift-dependent magnitude limit, MagLim. We account for the effect of magnification on both the flux and size selection of galaxies, accounting for systematic effects using the Balrog image simulations. We estimate the impact of magnification on the galaxy clustering and galaxy-galaxy lensing cosmology analysis, finding it to be a significant systematic for the MagLim sample. We show cosmological constraints from the galaxy clustering auto-correlation and galaxy-galaxy lensing signal with different magnifications priors, finding broad consistency in cosmological parameters in $\Lambda$CDM and $w$CDM. However, when magnification bias amplitude is allowed to be free, we find the two-point correlations functions prefer a different amplitude to the fiducial input derived from the image simulations. We validate the magnification analysis by comparing the cross-clustering between lens bins with the prediction from the baseline analysis, which uses only the auto-correlation of the lens bins, indicating systematics other than magnification may be the cause of the discrepancy. We show adding the cross-clustering between lens redshift bins to the fit significantly improves the constraints on lens magnification parameters and allows uninformative priors to be used on magnification coefficients, without any loss of constraining power or prior volume concerns. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09864-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09864) | **Development of the Low Frequency Telescope Focal Plane Detector Modules  for LiteBIRD**  |
|| Benjamin Westbrook, et al. -- incl., <mark>Adrian T. Lee</mark>, <mark>Michael J. Link</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *SPIE Astronomical Telescope + Instrumentation (AS22)*|
|**Abstract**| LiteBIRD is a JAXA-led strategic large-class satellite mission designed to measure the polarization of the cosmic microwave background and Galactic foregrounds from 34 to 448 GHz across the entire sky from L2 in the late 2020s. The scientific payload includes three telescopes which are called the low-, mid-, and high-frequency telescopes each with their own receiver that covers a portion of the mission's frequency range. The low frequency telescope will map synchrotron radiation from the Galactic foreground and the cosmic microwave background. We discuss the design, fabrication, and characterization of the low-frequency focal plane modules for low-frequency telescope, which has a total bandwidth ranging from 34 to 161 GHz. There will be a total of 4 different pixel types with 8 overlapping bands to cover the full frequency range. These modules are housed in a single low-frequency focal plane unit which provides thermal isolation, mechanical support, and radiative baffling for the detectors. The module design implements multi-chroic lenslet-coupled sinuous antenna arrays coupled to transition edge sensor bolometers read out with frequency-domain mulitplexing. While this technology has strong heritage in ground-based cosmic microwave background experiments, the broad frequency coverage, low optical loading conditions, and the high cosmic ray background of the space environment require further development of this technology to be suitable for LiteBIRD. In these proceedings, we discuss the optical and bolometeric characterization of a triplexing prototype pixel with bands centered on 78, 100, and 140 GHz. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09877-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09877) | **Revealing high-z Fermi-LAT BL Lacs using Swift and SARA data with  photometric analysis**  |
|| Y. Sheng, et al. -- incl., <mark>N. Torres-Alba</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *16pages, 7 figures*|
|**Abstract**| BL Lacertae (BL Lac) objects are one type of blazar, distinguished by their featureless optical spectrum. This presents a challenge in measuring the redshift of the BL Lacs. This paper uses the photometric dropout technique to measure the redshifts of BL Lac objects. Space-based telescope \emph{Swift} and ground-based SARA telescopes are employed to provide magnitudes in the $uvw2,\ uvm2,\ uvw1,\ u,\ b,\ v,\ g',\ r',\ i',\ z'$ filters. We observe 60 BL Lacs and report reliable redshift upper limits for 41 sources. We discover four high-$z$ BL Lacs ($z>1.3$), bringing the number of high-$z$ BL Lacs found by this method up to 20. We also discuss the blazar sequence, the \emph{Fermi} blazar divide, and the gamma-ray horizon using the 4LAC catalog and all high-$z$ BL Lacs discovered with the photo-z technique. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09763-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09763) | **The Einstein Probe Mission**  |
|| Weimin Yuan, <mark>Chen Zhang</mark>, Yong Chen, <mark>Zhixing Ling</mark> |
|*Appeared on*| *2022-09-21*|
|*Comments*| *29 pages, 23 figures; Invited chapter for Handbook of X-ray and Gamma-ray Astrophysics (Eds. C. Bambi and A. Santangelo, Springer Singapore, expected in 2022)*|
|**Abstract**| The Einstein Probe (EP) is a mission designed to monitor the sky in the soft X-ray band. It will perform systematic surveys and characterisation of high-energy transients and monitoring of variable objects at unprecedented sensitivity and monitoring cadences. It has a large instantaneous field-of-view (3,600 sq. deg.), that is realised via the lobster-eye micro-pore X-ray focusing optics. EP also carries a conventional X-ray focusing telescope with a larger effective area to perform followup observations and precise positioning of newly-discovered transients. Alerts for transient objects will be issued publicly and timely. The scientific goals of EP are concerned with discovering faint, distant or rare types of high-energy transients and variable sources. Within the confines of a modest-sized mission, EP will cover a wide range of scientific topics, from the nearby to high-redshift Universe. The Einstein Probe is a mission of the Chinese Academy of Sciences, and also an international collaborative project. This paper presents the background, scientific objectives, and the mission design including the micro-pore optics and CMOS technologies adopted, the instruments and their expected performance, and the mission profile. The development status of the project is also presented. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xd1 in position 10013: invalid continuation byte</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.09823.md
    + _build/html/tmp_2209.09823/./Dhouib_fig1.png
    + _build/html/tmp_2209.09823/./Dhouib_fig2.png
    + _build/html/tmp_2209.09823/./Dhouib_fig3.png
    + _build/html/tmp_2209.09823/./Dhouib_fig4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}{{\mathrm{km~s^{-1}}}}$
$\newcommand{\kpc}{{\mathrm{kpc}}}$
$\newcommand{\thefootnote}{\fnsymbol{footnote}}$
$\newcommand{\BibTeX}{{\rm B\kern-.05em{\sc i\kern-.025em b}$
$    T\kern-.1667em\lower.7ex\hbox{E}\kern-.125emX}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}{{\mathrm{km~s^{-1}}}}$
$\newcommand{$\kpc$}{{\mathrm{kpc}}}$
$\newcommand{$\thefootnote$}{\fnsymbol{footnote}}$
$\newcommand{\BibTeX}{{\rm B\kern-.05em{\sc i\kern-.025em b}$
$    T\kern-.1667em\lower.7ex\hbox{E}\kern-.125emX}}$</div>



<div id="title">

#  stars?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.09823-b31b1b.svg)](https://arxiv.org/abs/2209.09823)<mark>Appeared on: 2022-09-21</mark> - _4 pages, 2 figures. Proceeding of the Annual meeting of the French Society of Astronomy and Astrophysics (SF2A 2022)_

</div>
<div id="authors">

H. Dhouib, et al.

</div>
<div id="abstract">

**Abstract:** One of the major discoveries of asteroseismology is the signature of a strong extraction of angular momentum (AM) in the radiative zones of stars across the entire Hertzsprung-Russell diagram, resulting in weak core-to-surface rotation contrasts. Despite all efforts, a consistent AM transport theory, which reproduces both the internal rotation and mixing probed thanks to the seismology of stars, remains one of the major open problems in modern stellar astrophysics.A possible key ingredient to figure out this puzzle is magnetic field with its various possible topologies. Among them, strong axisymmetric toroidal fields, which are subject to the so-called Tayler MHD instability, could play a major role. They could trigger a dynamo action in radiative layers while the resulting magnetic torque allows an efficient transport of AM.But is it possible to detect signatures of these deep toroidal magnetic fields?The only way to answer this question is asteroseismology and the best laboratories of study are intermediate-mass and massive stars because of their external radiative envelope. Since most of these are rapid rotators during their main-sequence, we have to study stellar pulsations propagating in stably stratified, rotating, and potentially strongly magnetised radiative zones.For that, we generalise the traditional approximation of rotation, which provides in its classic version a flexible treatment of the adiabatic propagation of gravito-inertial modes, by taking simultaneously general axisymmetric differential rotation and toroidal magnetic fields into account.Using this new non-perturbative formalism, we derive the asymptotic properties of magneto-gravito-inertial modes and we explore the different possible field configurations. We found that the magnetic effects should be detectable for equatorial fields using high-precision asteroseismic data.

</div>

<div id="div_fig1">

<img src="tmp_2209.09823/./Dhouib_fig1.png" alt="Fig1.1" width="50%"/><img src="tmp_2209.09823/./Dhouib_fig2.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Equatorial {\bf (Left)} and hemispheric {\bf (Right)} toroidal magnetic configurations. (*fig:field*)

</div>
<div id="div_fig2">

<img src="tmp_2209.09823/./Dhouib_fig3.png" alt="Fig2.1" width="50%"/><img src="tmp_2209.09823/./Dhouib_fig4.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Detectability of the effect of an equatorial {\bf (Left)} and hemispheric {\bf (Right)} toroidal magnetic fields on the $\{k=0,m=1\}$ mode as a function of the radial order  $n$ using a $1.6{\rm M}_\odot$$\gamma $Dor model near ZAMS. (*fig:detect*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

39  publications in the last 7 days.
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers